In [1]:
from numba import njit

@njit
def gridlookup2(x0,xgrid):

    nx = np.shape(xgrid)[0]

    ix = 0
    for jx in range(nx):
        if x0<=xgrid[jx]:
            break
        ix = ix+1

    ix = min(max(1,ix),nx-1)

    return ix-1

@njit
def driver(r0,vmat0,mu0,Ge,Pe,knotsb,β,α,δ,znow,lnow,critin,critmu):

    vmat1 = np.zeros((nb,ne))
    gmat0 = np.zeros((nb,ne))

    # Factor Price
    m0 = lnow*(r0/α/znow)**(1/(α-1))
    w0 = (1-α)*znow*m0**(α)*lnow**(-α)

    # instanteneous utility
    umat0 = np.zeros((nb,nb,ne))
    for ie in range(ne):
        for ib in range(nb):
            for jb in range(nb):

                enow = Ge[ie]
                know = knotsb[ib]
                kp   = knotsb[jb]
                cnow = w0*enow + (1.0+r0-δ)*know - kp;

                if cnow>0:
                    umat0[jb,ib,ie] = np.log(cnow)
                else:
                    umat0[jb,ib,ie] = -1e+10

    # value function iteration
    diff   = 1e+4
    iterin = 0

    while diff>critin:

        # NOTE: np.max with axis argument not supported by numba
#         for ie in range(ne):

#             utemp = umat0[:,:,ie]
#             vcond = Pe[ie,0]*vmat0[:,0] + Pe[ie,1]*vmat0[:,1]
#             vtemp = utemp + β*vcond.reshape(nb,1) # BUG: vcond should have been (nbx1)
#             vmat1[:,ie] = np.max(vtemp,axis=0)

        for ie in range(ne):

            for ib in range(nb):

                utemp = umat0[:,ib,ie]
                vcond = Pe[ie,0]*vmat0[:,0] + Pe[ie,1]*vmat0[:,1]
                vtemp = utemp + β*vcond
                vmat1[ib,ie] = np.max(vtemp)

        diff = np.max(np.abs(vmat1-vmat0))
        iterin = iterin+1
        vmat0 = np.copy(vmat1)
#         print([iterin,diff])

    # policy function
    for ie in range(ne):

        for ib in range(nb):

            utemp = umat0[:,ib,ie]
            vcond = Pe[ie,0]*vmat0[:,0] + Pe[ie,1]*vmat0[:,1]
            vtemp = utemp + β*vcond
            jb = np.argmax(vtemp)
            gmat0[ib,ie] = knotsb[jb] 

    # transition matrix
    AA = np.zeros((nb*ne,nb*ne))
    wb = np.zeros((nb,ne))

    for ie in range(ne):

        for ib in range(nb):

            know = knotsb[ib]
            kp = gmat0[ib,ie]
            kb = gridlookup2(kp,knotsb)
            wb[ib,ie] = (knotsb[kb+1]-kp)/(knotsb[kb+1]-knotsb[kb])

            for je in range(ne):

                ia = nb*ie+ib
                ja = nb*je+kb
                AA[ia,ja]   = wb[ib,ie]*Pe[ie,je]
                AA[ia,ja+1] = (1.0-wb[ib,ie])*Pe[ie,je]

    # distribution
    diffmu = 1e+4
    dist   = np.zeros(2*nb)
    mu1    = np.zeros((nb,ne))

    while diffmu>critmu:

        dist[0:nb]    = mu0[:,0]
        dist[nb:2*nb] = mu0[:,1]
#         dist = np.reshape(mu0,(2*nb,1),order='F') # NOTE: cannot use with numba
        dist = AA.T@dist
        mu1[:,0] = dist[0:nb]
        mu1[:,1] = dist[nb:2*nb]
#         mu1  = np.reshape(dist,(nb,2),order='F')

        diffmu = np.max(np.abs(mu1-mu0))
        mu0  = mu1/np.sum(mu1)
        
    # Calculate K
    m1 = 0.0
    for ie in range(ne):

#         m1 = m1 + mu0[:,ie].T@knotsb
        m1 = m1 + np.dot(mu0[:,ie],knotsb)

    r1 = (α)*znow*m1**(α-1)*lnow**(1-α)
        
    return r1, vmat0, mu0

In [2]:
# Bewley-Aiyagari model
# May 2020, Takeki Sunakawa
import numpy as np
import matplotlib.pyplot as plt
import time

β  = 0.96
α  = 0.36 
δ  = 1-0.92
σ  = 1.5
μ  = 0.05
pee = 0.925
puu = 0.5

# grid points
ne = 2
Ge = np.array([1.0,μ]) 
Pe = np.array([[pee,1-pee],
    [1-puu,puu]]) 

nb = 1001
kmin = 0
kmax = 20.0
knotsb = np.linspace(kmin,kmax,nb)

mue = np.identity(ne)
for i in range(10000):
    mue = Pe@mue
    
mue = mue[0,:]
znow = 1.0
# efficiency unit of labor
lnow = Ge.T@mue

# initial distribution
vmat0 = np.zeros((nb,ne))
mu0   = np.ones((nb,ne))/(nb*ne)

# initial value of r
mnow = lnow*(α*β/(1.0-β*(1.0-δ)))**(1.0/(1.0-α))
mnow = 5.2074 # from My_Aiyagari.m
m0 = mnow
r0 = (α)*znow*mnow**(α-1)*lnow**(1-α);


start = time.time()
critin  = 1e-4
critmu  = 1e-8
critout = 1e-3
diffout = 1e+3
damp = 0.01
iter = 0

# for test
# r1,vmat0,mu0 = driver(r0,vmat0,mu0,Ge,Pe,knotsb,β,α,δ,znow,lnow,critin,critmu)

while diffout>critout:

    r1,vmat0,mu0 = driver(r0,vmat0,mu0,Ge,Pe,knotsb,β,α,δ,znow,lnow,critin,critmu)

    diffout = np.abs(np.log(r1)-np.log(r0))
    iter = iter+1
    print("iter = %4d, diff = %5.6f, oldr = %5.6f, newr = %5.6f" % (iter,diffout,r0,r1))

    # Update K
    r0 = damp*r1 + (1.0-damp)*r0

eptime = time.time()-start
print("Elapsed time is %5.10f." % eptime)

<ipython-input-1-0e677354c5b3>:126: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, C))
  m1 = m1 + np.dot(mu0[:,ie],knotsb)


iter =    1, diff = 0.191966, oldr = 0.115052, newr = 0.139401
iter =    2, diff = 0.179063, oldr = 0.115296, newr = 0.137905
iter =    3, diff = 0.166392, oldr = 0.115522, newr = 0.136435
iter =    4, diff = 0.153681, oldr = 0.115731, newr = 0.134956
iter =    5, diff = 0.142776, oldr = 0.115923, newr = 0.133714
iter =    6, diff = 0.131264, oldr = 0.116101, newr = 0.132387
iter =    7, diff = 0.121640, oldr = 0.116264, newr = 0.131303
iter =    8, diff = 0.111574, oldr = 0.116414, newr = 0.130156
iter =    9, diff = 0.102348, oldr = 0.116552, newr = 0.129112
iter =   10, diff = 0.093758, oldr = 0.116677, newr = 0.128146
iter =   11, diff = 0.086373, oldr = 0.116792, newr = 0.127328
iter =   12, diff = 0.078227, oldr = 0.116897, newr = 0.126409
iter =   13, diff = 0.071472, oldr = 0.116993, newr = 0.125660
iter =   14, diff = 0.064705, oldr = 0.117079, newr = 0.124905
iter =   15, diff = 0.058778, oldr = 0.117158, newr = 0.124250
iter =   16, diff = 0.053440, oldr = 0.117228, newr = 0